# Kepler Exoplanet Search Results
## 10000 exoplanet candidates examined by the Kepler Space Observatory
### [NASA](https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=cumulative)

## Data Dictionary 

|column_names|definition|
|------------|----------|
|kepid|KepID|
|kepoi_name|KOI Name|
|kepler_name|Kepler Name|
|koi_disposition|Exoplanet Archive Disposition|
|koi_pdisposition|Disposition Using Kepler Data|
|koi_score|Disposition Score|
|koi_fpflag_nt|Not Transit-Like False Positive Flag|
|koi_fpflag_ss|Stellar Eclipse False Positive Flag|
|koi_fpflag_co|Centroid Offset False Positive Flag|
|koi_fpflag_ec|Ephemeris Match Indicates Contamination False Positive Flag|
|koi_period|Orbital Period [days]|
|koi_period_err1|Orbital Period Upper Unc. [days]|
|koi_period_err2|Orbital Period Lower Unc. [days]|
|koi_time0bk|Transit Epoch [BKJD]|
|koi_time0bk_err1|Transit Epoch Upper Unc. [BKJD]|
|koi_time0bk_err2|Transit Epoch Lower Unc. [BKJD]|
|koi_impact|Impact Parameter|
|koi_impact_err1|Impact Parameter Upper Unc.|
|koi_impact_err2|Impact Parameter Lower Unc.|
|koi_duration|Transit Duration [hrs]|
|koi_duration_err1|Transit Duration Upper Unc. [hrs]|
|koi_duration_err2|Transit Duration Lower Unc. [hrs]|
|koi_depth|Transit Depth [ppm]|
|koi_depth_err1|Transit Depth Upper Unc. [ppm]|
|koi_depth_err2|Transit Depth Lower Unc. [ppm]|
|koi_prad|Planetary Radius [Earth radii]|
|koi_prad_err1|Planetary Radius Upper Unc. [Earth radii]|
|koi_prad_err2|Planetary Radius Lower Unc. [Earth radii]|
|koi_teq|Equilibrium Temperature [K]|
|koi_teq_err1|Equilibrium Temperature Upper Unc. [K]|
|koi_teq_err2|Equilibrium Temperature Lower Unc. [K]|
|koi_insol|Insolation Flux [Earth flux]|
|koi_insol_err1|Insolation Flux Upper Unc. [Earth flux]|
|koi_insol_err2|Insolation Flux Lower Unc. [Earth flux]|
|koi_model_snr|Transit Signal-to-Noise|
|koi_tce_plnt_num|TCE Planet Number|
|koi_tce_delivname|TCE Delivery|
|koi_steff|Stellar Effective Temperature [K]|
|koi_steff_err1|Stellar Effective Temperature Upper Unc. [K]|
|koi_steff_err2|Stellar Effective Temperature Lower Unc. [K]|
|koi_slogg|Stellar Surface Gravity [log10(cm/s**2)]|
|koi_slogg_err1|Stellar Surface Gravity Upper Unc. [log10(cm/s**2)]|
|koi_slogg_err2|Stellar Surface Gravity Lower Unc. [log10(cm/s**2)]|
|koi_srad|Stellar Radius [Solar radii]|
|koi_srad_err1|Stellar Radius Upper Unc. [Solar radii]|
|koi_srad_err2|Stellar Radius Lower Unc. [Solar radii]|
|ra|RA [decimal degrees]|
|dec|Dec [decimal degrees]|
|koi_kepmag|Kepler-band [mag]|

In [26]:
import pandas as pd
import numpy as np
import matplotlib as plt
from category_encoders import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier

# pandas display options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Wrangle

In [3]:
def wrangle(filepath):
    # read df
    df = pd.read_csv(filepath).set_index('kepid')
    
    # drop cols with < 3000 null values
    df.dropna(axis=1, thresh=3000, inplace=True)
    
    # drop high cardinality cols 
    cols = [col for col in df.select_dtypes('object').columns
            if df[col].nunique() > 100]
    df.drop(columns=cols, inplace=True)
    
    # drop repeat rows (observations)
    df.drop_duplicates(inplace=True)
    
    # drop cols to prevent leakage
    df.drop(columns=['koi_pdisposition', 'koi_score'], inplace=True)
    
    return df

FILE_PATH = 'data/kep_observations.csv'

df = wrangle(FILE_PATH)

In [4]:
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
kepid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10797460,CONFIRMED,0,0,0,0,9.488036,2.780000e-05,-2.780000e-05,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,616.0,19.5,-19.5,2.26,0.26,-0.15,793.0,93.59,29.45,-16.65,35.8,1.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
10797460,CONFIRMED,0,0,0,0,54.418383,2.480000e-04,-2.480000e-04,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,875.0,35.5,-35.5,2.83,0.32,-0.19,443.0,9.11,2.87,-1.62,25.8,2.0,q1_q17_dr25_tce,5455.0,81.0,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
10811496,CANDIDATE,0,0,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,10800.0,171.0,-171.0,14.60,3.92,-1.31,638.0,39.30,31.04,-10.49,76.3,1.0,q1_q17_dr25_tce,5853.0,158.0,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
10848459,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,8080.0,12.8,-12.8,33.46,8.50,-2.83,1395.0,891.96,668.95,-230.35,505.6,1.0,q1_q17_dr25_tce,5805.0,157.0,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
10854555,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,603.0,16.9,-16.9,2.75,0.88,-0.35,1406.0,926.16,874.33,-314.24,40.9,1.0,q1_q17_dr25_tce,6031.0,169.0,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


# Split

In [49]:
target = 'koi_disposition'
# target vector
y = df[target]
# feature matrix
X = df.drop(columns=target)

# 80/20 train validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# reset indices for clasification report
X_train.reset_index(inplace=True)
X_val.reset_index(inplace=True)

# Baseline 

In [43]:
# majority class
print('Baseline Accuracy:', y_train.value_counts(normalize=True).max())

Baseline Accuracy: 0.5098679911122729


# Build Models

## Gradient Boosted Tree

In [44]:
xgb = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    XGBClassifier(n_estimators=90,
                  max_depth=2,
                  random_state=42, 
                  n_jobs=-1)
)

xgb.fit(X_train, y_train);

/home/austin/.local/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:58:56] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


## Multinomial Logestic Regression

In [45]:
mlr = make_pipeline(
    OneHotEncoder(),
    SimpleImputer(),
    LogisticRegression(multi_class='multinomial',
                       solver='sag',
                       max_iter=10000,
                       random_state=42)
)

mlr.fit(X_train, y_train);

/home/austin/.local/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


# Communicate Results 

## Gradient Boosted Tree Accuracy

In [46]:
print('XGB Training Accuracy:', xgb.score(X_train, y_train))
print('XGB Validation Accuracy:', xgb.score(X_val, y_val))

XGB Training Accuracy: 0.9345183636125997
XGB Validation Accuracy: 0.909566126502875


## Multinomial Logestic Regression Accuracy

In [47]:
print('MLR Training Accuracy:', mlr.score(X_train, y_train))
print('MLR Validation Accuracy:', mlr.score(X_val, y_val))

MLR Training Accuracy: 0.5477715331329238
MLR Validation Accuracy: 0.5279665446941976


## Classification Report

In [56]:
print('-------------------------------------------------------')
print('Gradient Boosted Tree'.upper())
print('-------------------------------------------------------')
print(classification_report(y_train, xgb.predict(X_train)))
print()
print('-------------------------------------------------------')
print('Multinomial Logestic Regression'.upper())
print('-------------------------------------------------------')
print(classification_report(y_train, mlr.predict(X_train)))

-------------------------------------------------------
GRADIENT BOOSTED TREE
-------------------------------------------------------
                precision    recall  f1-score   support

     CANDIDATE       0.89      0.84      0.87      1882
     CONFIRMED       0.85      0.91      0.88      1868
FALSE POSITIVE       0.99      0.99      0.99      3901

      accuracy                           0.93      7651
     macro avg       0.91      0.91      0.91      7651
  weighted avg       0.94      0.93      0.93      7651


-------------------------------------------------------
MULTINOMIAL LOGESTIC REGRESSION
-------------------------------------------------------
                precision    recall  f1-score   support

     CANDIDATE       1.00      0.00      0.00      1882
     CONFIRMED       0.40      0.39      0.40      1868
FALSE POSITIVE       0.59      0.89      0.71      3901

      accuracy                           0.55      7651
     macro avg       0.67      0.43      0.3